<a href="https://colab.research.google.com/github/siren7075/OpenAI_Embeddings_Public/blob/main/OpenAI_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Open AI

In [ ]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
# imports
from typing import List

import pandas as pd
import pickle
import ast

from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

In [ ]:
openai.api_key = ""

In [ ]:
df = pd.read_csv("/content/Cusips_Combined.csv")
df.head(5)

,cusip,security_name,coupon,maturity_date,issuer,instrument_type_gps_name,industry_group_lehman2_name,sector_lehman2_name,currency_code,country,rating_moody_or_sp,liquidity,bids,asks,combined
0,000000299,S&P 400 MIDCAP INDEX,NaN,NaN,SPXT,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"S&P 400 MIDCAP INDEX,NULL,NULL,SPXT,EMI,UNKNOW..."
1,000000711,S&P 500 INDEX,0.0,NaN,SPXT,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"S&P 500 INDEX,0,NULL,SPXT,EMI,UNKNOWN,UNKNOWN,..."
2,000000737,DOW JONES INDUS. AVG,0.0,NaN,INDU,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"DOW JONES INDUS. AVG,0,NULL,INDU,EMI,UNKNOWN,U..."
3,000001KP1,RUSSELL 1000 INDEX,0.0,NaN,RLG,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"RUSSELL 1000 INDEX,0,NULL,RLG,EMI,UNKNOWN,UNKN..."
4,000001KR7,RUSSEL 3000 INDEX,0.0,NaN,RLG,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR,NaN,NaN,NaN,"RUSSEL 3000 INDEX,0,NULL,RLG,EMI,UNKNOWN,UNKNO..."


In [ ]:
len(df)

10102

In [ ]:
df_1000 = df.head(1000)
len(df_1000)

1000

In [ ]:
df['davinci_similarity'] = df_1000.combined.apply(lambda x: get_embedding(x, engine='text-similarity-davinci-001'))
df.to_csv('/Cusips_Embeddings.csv')

In [9]:
df_embeddings = pd.read_csv("/content/Cusips_Embeddings.csv")
df_embeddings_1000 = df_embeddings.head(1000)

In [13]:
# get embeddings for all strings
embeddings = df_embeddings_1000.davinci_similarity
embeddings_clean = [ast.literal_eval(i) for i in embeddings]

In [15]:
query_embedding = embeddings_clean[0]

In [17]:
# get distances between the source embedding and other embeddings (function from embeddings_utils.py)
distances = distances_from_embeddings(query_embedding, embeddings_clean, distance_metric="cosine")
# get indices of nearest neighbors (function from embeddings_utils.py)
indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)

In [18]:
combined_strings = df_embeddings_1000.combined.values.tolist()
query_string = combined_strings[0]
print(f"Source string: {query_string}")

Source string: S&P 400 MIDCAP INDEX,NULL,NULL,SPXT,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR


In [19]:
# print out its k nearest neighbors, defaults to 5
k_counter = 0
for i in indices_of_nearest_neighbors:
# skip any strings that are identical matches to the starting string
    if query_string == combined_strings[i]:
        continue
    # stop after printing out k articles
    if k_counter >= 5:
        break
    k_counter += 1

    # print out the similar strings and their distances
    print(
        f"""
    --- Recommendation #{k_counter} (nearest neighbor {k_counter} of 5 ---
    String: {combined_strings[i]}
    Distance: {distances[i]:0.3f}"""
    )


    --- Recommendation #1 (nearest neighbor 1 of 5 ---
    String: S&P 500 INDEX,0,NULL,SPXT,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR
    Distance: 0.025

    --- Recommendation #2 (nearest neighbor 2 of 5 ---
    String: DOW JONES INDUS. AVG,0,NULL,INDU,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR
    Distance: 0.049

    --- Recommendation #3 (nearest neighbor 3 of 5 ---
    String: RUSSELL 1000 INDEX,0,NULL,RLG,EMI,UNKNOWN,UNKNOWN,USD,UNITED STATES,NR
    Distance: 0.058

    --- Recommendation #4 (nearest neighbor 4 of 5 ---
    String: INVESCO COMSTOCK FD-A,NULL,NULL,IVZ,INVCOSTK,TRUSTS AND MUTUAL FUNDS,TRUSTS AND FUNDS,USD,UNITED STATES,NR
    Distance: 0.073

    --- Recommendation #5 (nearest neighbor 5 of 5 ---
    String: AFC GAMMA INC,NULL,NULL,AFCG,REIT,INDUSTRIAL,CORPORATE,USD,UNITED STATES,NR
    Distance: 0.073
